In [ ]:
import random as rd
import pandas as pd
import math
import numpy as np
import random

from itertools import chain, combinations

In [ ]:
masterTable = pd.read_csv('data.csv')

In [ ]:
masterTable.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type,Unknown 1,Unknown 2,Unknown 3
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


In [ ]:
dataset = masterTable.iloc[: , :10]

In [ ]:
dataset.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points
0,2596,51,3,258,0,510,221,232,148,6279
1,2590,56,2,212,-6,390,220,235,151,6225
2,2804,139,9,268,65,3180,234,238,135,6121
3,2785,155,18,242,118,3090,238,238,122,6211
4,2595,45,2,153,-1,391,220,234,150,6172


In [ ]:
def generate_subsets(arr):
    all_subsets = []
    for r in range(1, len(arr) + 1):
        subsets_r = list(combinations(arr, r))
        all_subsets.extend(subsets_r)

    return [list(subset) for subset in all_subsets]

In [ ]:
def genr_data_distr(col,count):
  l_data = []
  col =  col.to_list()
  for i in range(count):
    index = rd.choice(col)
    l_data.append([index])
  return l_data

In [ ]:
def getRanges(cols, noSamplesSet):
  attToDist = dict()
  for c in cols:
    attToDist[c] = genr_data_distr(dataset[c], noSamplesSet)
  return attToDist

In [ ]:
def create_zero_dataframe(n, D):
    data = {f"Attribute_{i}": [0] * n for i in range(1, D+1)}
    df = pd.DataFrame(data)
    return df

def generateSkeletonData(n, columns):
  zero_df = create_zero_dataframe(n, 4*len(columns))
  newColNames =[]
  for c in columns:
    newColNames.append(c + " <")
    newColNames.append(c + " =")
    newColNames.append(c + " >")
    newColNames.append(c + " values")
  zero_df.columns = newColNames
  return zero_df

In [ ]:
def findIndex(col, columns):
  for i,c in enumerate(columns):
    if (c == col):
      return i

In [ ]:
def generate_random_ohe(n):
    possibilities = [[1, 1, 0], [0, 1, 1], [1, 0, 0], [0, 1, 0], [0, 0, 1]]
    samples = []
    for _ in range(n):
        sample = random.choice(possibilities)
        samples.append(sample)
    return samples


In [ ]:
n = 20000
columns = list(dataset.columns)
subsets = generate_subsets(columns)

sampleSize = math.ceil(n/len(subsets))
distributions = getRanges(columns, sampleSize)
data = generateSkeletonData(sampleSize*len(subsets), columns)
ctr = 0
for s in subsets:
  for c in s:
    index = findIndex(c, columns)
    randoperators = generate_random_ohe(sampleSize)
    data.iloc[ctr : ctr+sampleSize, 4*index +3 : 4*index+4] = distributions[c]
    data.iloc[ctr : ctr+sampleSize, 4*index : 4*index+3] = randoperators
  ctr = ctr + sampleSize
